This code similates an IRS (Interest Rape Swap) instrument:

Check also:
*ql.DiscountingSwapEngine

*ql.VanillaSwap

*Quantlib Cookbook: 12. Simulating Interest Rates using Hull
White Model

http://gouthamanbalaraman.com/blog/interest-rate-swap-quantlib-python.html
https://quantlib-python-docs.readthedocs.io/en/latest/examples/fixedincome/vanillaswap.html

-------------------------------------------------------------------------------
https://quantlib-python-docs.readthedocs.io/en/latest/instruments.html
https://quant.stackexchange.com/questions/44712/quantlib-python-use-zero-rates-to-get-the-originally-bootstrapped-curve
http://gouthamanbalaraman.com/blog/quantlib-bond-modeling.html
http://gouthamanbalaraman.com/blog/hull-white-simulation-quantlib-python.html

In [ ]:
# <!-- collapse=True -->
# pip install QuantLib-Python
import QuantLib as ql
calculation_date = ql.Date(14, 1, 2020)
ql.Settings.instance().evaluationDate = calculation_date

In [ ]:
yts = ql.RelinkableYieldTermStructureHandle()

instruments = [
    ('depo', '6M', 0.025),
    ('fra', '6M', 0.03),
    ('swap', '1Y', 0.031),
    ('swap', '2Y', 0.032),
    ('swap', '3Y', 0.035)
]

helpers = ql.RateHelperVector()
index = ql.Euribor6M(yts)
for instrument, tenor, rate in instruments:
    if instrument == 'depo':
        helpers.append( ql.DepositRateHelper(rate, index) )
    if instrument == 'fra':
        monthsToStart = ql.Period(tenor).length()
        helpers.append( ql.FraRateHelper(rate, monthsToStart, index) )
    if instrument == 'swap':
        swapIndex = ql.EuriborSwapIsdaFixA(ql.Period(tenor))
        helpers.append( ql.SwapRateHelper(rate, swapIndex))
curve = ql.PiecewiseLogCubicDiscount(2, ql.TARGET(), helpers, ql.ActualActual())


In [ ]:
yts.linkTo(curve)
engine = ql.DiscountingSwapEngine(yts)


In [ ]:
tenor = ql.Period('2y')
fixedRate = 0.05
forwardStart = ql.Period("2D")

swap = ql.MakeVanillaSwap(tenor, index, fixedRate, forwardStart, Nominal=10e6, pricingEngine=engine)


In [ ]:
fairRate = swap.fairRate()
npv = swap.NPV()

print(f"Fair swap rate: {fairRate:.3%}")
print(f"Swap NPV: {npv:,.3f}")

In [ ]:
import pandas as pd
pd.options.display.float_format = "{:,.2f}".format

cashflows = pd.DataFrame({
    'date': cf.date(),
    'amount': cf.amount()
    } for cf in swap.leg(1))
display(cashflows)

In [ ]:
cashflows = pd.DataFrame({
    'nominal': cf.nominal(),
    'accrualStartDate': cf.accrualStartDate().ISO(),
    'accrualEndDate': cf.accrualEndDate().ISO(),
    'rate': cf.rate(),
    'amount': cf.amount()
    } for cf in map(ql.as_coupon, swap.leg(1)))
display(cashflows)